In [ ]:
!pip install import-ipynb

In [ ]:
import import_ipynb
import pandas as pd
import config
import requests
import matplotlib.pyplot as plt
import csv

In [ ]:
import BaseApiClassImplicitGrantFlow as ba

In [ ]:
client_id = config.client_id
client_secret = config.client_secret

In [ ]:
spotify = ba.SpotifyAPI(client_id, client_secret);
spotify.set_scope("playlist-read-private")
spotify.perform_auth_grant_flow()

In [ ]:
spotify.parse_redirected_url()
spotify.set_access_token_and_expire()
spotify.set_headers(spotify.get_access_token())

In [ ]:
playlist_json = spotify.get_current_user_playlists();
playlist_ids = spotify.get_playlists_ids(playlist_json)
artists_and_genres = spotify.get_all_artists_from_playlists(playlist_ids)

In [ ]:
artists_and_genres_merged = sum(artists_and_genres, [])

In [ ]:
w = csv.writer(open("artists_genres.csv", "w"), delimiter=',')
w.writerow(['artist', 'genres'])
for record in artists_and_genres_merged:
    for key, val in record.items():
        val = ','.join(val)
        w.writerow([key, val])

In [ ]:
songs_data = pd.read_csv('artists_genres.csv')

In [ ]:
genres_df = songs_data['genres'].str.split(',', expand=True).rename(columns = lambda x: "genre"+str(x+1))

In [ ]:
genres_list = []
for col in genres_df:
    genres_list.append(genres_df[col].unique().tolist())

#removing duplicates
genres_list = list(set(sum(genres_list, [])))

In [ ]:
def remove_none_elements_from_list(list):
    return [e for e in list if e != None]

genres_list = remove_none_elements_from_list(genres_list)

In [ ]:
occurences = {}

for genre in genres_list:
    count = 0
    for genre_col in genres_df:
        count = count + (genres_df[genre_col] == genre).sum()
    occurences[genre] = count
    
sorted_occurences = {}
sorted_keys = sorted(occurences, key=occurences.get)

for key in sorted_keys:
    sorted_occurences[key] = occurences[key]

In [ ]:
plt.figure(figsize=(100,100))
plt.xticks(rotation='vertical')
plt.bar(*zip(*sorted_occurences.items()))
plt.show
plt.savefig('result.png')